In [ ]:
!pip install pyomo
import pyomo.environ as pyo
from pyomo.opt import SolverManagerFactory
import pandas as pd
import math

In [4]:
model = pyo.ConcreteModel()

model.i = pyo.RangeSet(1,15)

W = pd.read_excel('S7P2_Data.xlsx', sheet_name = 'Sheet1', header=0, index_col=0, usecols='A:B')
model.Pload = pyo.Param(initialize = 300)
Pload=model.Pload

model.Pmax = pyo.Param(initialize = 1000)
Pmax=model.Pmax

model.defml = pyo.Param(initialize = 6)
defml=model.defml

model.deflpm = pyo.Param(initialize = 1.25)
deflpm=model.deflpm

model.Lmax = pyo.Param(initialize = 14)
Lmax=model.Lmax

model.Dcm = pyo.Param(initialize = 3)
Dcm=model.Dcm

model.dWm = pyo.Param(initialize = 0.2)
dWm=model.dWm

model.S = pyo.Param(initialize = 234440)
S=model.S

model.G = pyo.Param(initialize = 11600000)
G=model.G

model.Dc = pyo.Var(domain = pyo.NonNegativeReals, bounds = (0.4,3))
Dc=model.Dc

model.dW = pyo.Var(domain = pyo.NonNegativeReals, bounds = (0.2,None))
dW=model.dW

model.defl = pyo.Var(domain = pyo.NonNegativeReals, bounds = (0.0018,6))
defl=model.defl

model.C = pyo.Var(domain = pyo.NonNegativeReals, bounds = (3,None))
C=model.C

model.K = pyo.Var(domain = pyo.NonNegativeReals, bounds = (None,560))
K=model.K

model.N = pyo.Var(domain = pyo.NonNegativeIntegers, bounds = (1,None))
N=model.N

model.x = pyo.Var(model.i, domain = pyo.Binary)
x=model.x

def objective_rule(model):
  return math.pi*Dc*(dW**2)*(N+2)/4
model.obj = pyo.Objective(rule = objective_rule, sense = pyo.minimize)

def con1(model):
  return Dc/dW == C
model.con1 = pyo.Constraint(rule = con1)

def con2(model):
  return ((4*C - 1)/(4*C - 4)) + 0.615/C == K
model.con2 = pyo.Constraint(rule = con2)

def con3(model):
  return (8*K*Pmax*Dc/(math.pi*dW**3)) <= S
model.co3 = pyo.Constraint(rule = con3)

def con4(model):
  return 8*(Dc**3)*N/(G*(dW**2)) == defl
model.con4 = pyo.Constraint(rule = con4)

def con5(model):
  return Pmax*defl + 1.05*(N + 2)*dW <= Lmax
model.con5 = pyo.Constraint(rule = con5)

def con6(model,i):
  return sum(W['D'][i]*x[i] for i in model.i) == dW
model.con6 = pyo.Constraint(rule = con6)

def con7(model, i):
  return sum(x[i] for i in model.i) == 1
model.con7 = pyo.Constraint(rule = con7)

In [ ]:
import os
os.environ['NEOS_EMAIL']=''
solver = SolverManagerFactory('neos') #SolverManagerFactory should be imported instead of SolverFactory
result = solver.solve(model,opt='minlp')
print(result)
print('objective function =', model.obj())
print('Number of coils=', N())
print('coils spring diameter is=', Dc())
print('wire diameter is=', dW())
for i in model.i:
  print('Wire Type',i,'is selected', x[i]() == 1)